# Spark DataFrame Genel Bakış
Bir projede sprak çalışmak istiyorsak bir session oluşturmalıyız.

SparkSession, Spark 2.0'dan itibaren kullanılan bir giriş noktasıdır. SparkSession, SparkContext, SQLContext ve HiveContext'leri birleştirir. SparkSession, DataFrame ve Dataset API'lerini kullanarak veri işleme işlemlerini gerçekleştirmek için kullanılır.

Özetle, SparkSession, Spark uygulamalarını başlatmak ve Spark işlevlerini kullanmak için kullanılan bir giriş noktasıdır.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PysparkExample").getOrCreate()

Tıpkı pandas daki gibi spark dataframe oluşturabiliriz.

Bu bir liste, dict veya pandas dataframe olabilir.

In [0]:
veri = [("Yusuf", "Gzb"), ("Hasan", "Aydoğmuş"), ("Merve", "Aka"), ("Sencer", "Cakız")]

df = spark.createDataFrame(veri, ["Ad", "Soyad"])
df.show()

+------+--------+
|    Ad|   Soyad|
+------+--------+
| Yusuf|     Gzb|
| Hasan|Aydoğmuş|
| Merve|     Aka|
|Sencer|   Cakız|
+------+--------+



Eğer bir kolonun ismini değiştirmek istersek ``withColumnRenamed`` fonksiyonunu kullanabiliriz.

In [0]:
df = df.withColumnRenamed("Ad","İsim")
df.show()

+------+--------+
|  İsim|   Soyad|
+------+--------+
| Yusuf|     Gzb|
| Hasan|Aydoğmuş|
| Merve|     Aka|
|Sencer|   Cakız|
+------+--------+



In [0]:
veri = [{"ad": "Yusuf", "soyad": "Gzb"}, {"ad": "Hasan", "soyad": "Aydoğmuş"},
        {"ad": "Merve", "soyad": "Aka"}, {"ad": "Sencer", "soyad": "Cakız"}]
df = spark.createDataFrame(veri)
df.show()

+------+--------+
|    ad|   soyad|
+------+--------+
| Yusuf|     Gzb|
| Hasan|Aydoğmuş|
| Merve|     Aka|
|Sencer|   Cakız|
+------+--------+



Verinin Tipi süreçlerimiz için çok önemlidir

In [0]:
type(df)

pyspark.sql.dataframe.DataFrame

## Spark ile veri okuma yöntemleri

#### 1. okuma yöntemi

`.format` Okuyacağımız verinin formatı -> CSV, Parquet, Json

`.option` Okuduğumuz formata göre yapacağımız ayarlamalar 

`.load` Okuyacağımız dosyanın tam dosya yolu ( Bu kısımda okuduğunuz alan çok önemlidir eğer hdfs de okuyorsanız hdfs deki tam yolu dbfs de okuyorsanız oranın tam yolunu belirtmeniz gerekmektedir)

In [0]:
df = spark.read.format("csv").option("header","True").load("dbfs:/FileStore/tables/grades.csv")
df.show()

+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|Last name|    "First name"|               "SSN"|        "Test1"| "Test2"| "Test3"| "Test4"| "Final"| "Grade"|
+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|  Alfalfa|      "Aloysius"|       "123-45-6789"|           40.0|    90.0|   100.0|    83.0|    49.0|    "D-"|
|   Alfred|    "University"|       "123-12-1234"|           41.0|    97.0|    96.0|    97.0|    48.0|    "D+"|
|    Gerty|        "Gramma"|       "567-89-0123"|           41.0|    80.0|    60.0|    40.0|    44.0|     "C"|
|  Android|      "Electric"|       "087-65-4321"|           42.0|    23.0|    36.0|    45.0|    47.0|    "B-"|
|  Bumpkin|          "Fred"|       "456-78-9012"|           43.0|    78.0|    88.0|    77.0|    45.0|    "A-"|
|   Rubble|         "Betty"|       "234-56-7890"|           44.0|    90.0|    80.0|    90.0|    46.0|    "C-"|
|

Okudumuz verinin nasıl bir yapıya sahip olduğunu görmek için `printSchema` kullanılır.

Aşağıda da görüldüğü gibi hepsi string çünkü herhangi bir standart belirmediğimiz için en kolay okuma yöntemi olan stringi tercih edecektir bu okurken hızlı okumasına ama veri üzerinde işlemler yapmamız gerektiğinde de zorlayıcı olacaktır

In [0]:
df.printSchema()

root
 |-- Last name: string (nullable = true)
 |--  "First name": string (nullable = true)
 |--  "SSN": string (nullable = true)
 |--         "Test1": string (nullable = true)
 |--  "Test2": string (nullable = true)
 |--  "Test3": string (nullable = true)
 |--  "Test4": string (nullable = true)
 |--  "Final": string (nullable = true)
 |--  "Grade": string (nullable = true)



String okuma durumunu çözmek için en basit yöntem (Önermediğimiz yöntem):

`inferSchema` True olarak ayarlanaktır 

Bu durum veriyi daha doğru tanıyacaktır ama okurken yavaşlık getirecektir okuduğu verinin boyutuna göre

In [0]:
df = spark.read.format("csv").option("header","True").option("inferSchema","True").load("dbfs:/FileStore/tables/grades.csv")
df.show()
df.printSchema()

+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|Last name|    "First name"|               "SSN"|        "Test1"| "Test2"| "Test3"| "Test4"| "Final"| "Grade"|
+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|  Alfalfa|      "Aloysius"|       "123-45-6789"|           40.0|    90.0|   100.0|    83.0|    49.0|    "D-"|
|   Alfred|    "University"|       "123-12-1234"|           41.0|    97.0|    96.0|    97.0|    48.0|    "D+"|
|    Gerty|        "Gramma"|       "567-89-0123"|           41.0|    80.0|    60.0|    40.0|    44.0|     "C"|
|  Android|      "Electric"|       "087-65-4321"|           42.0|    23.0|    36.0|    45.0|    47.0|    "B-"|
|  Bumpkin|          "Fred"|       "456-78-9012"|           43.0|    78.0|    88.0|    77.0|    45.0|    "A-"|
|   Rubble|         "Betty"|       "234-56-7890"|           44.0|    90.0|    80.0|    90.0|    46.0|    "C-"|
|

##### 2. Okuma Yöntemi

Diğer bir okuma yöntemi de format da dosya formatını belirtmek yerine direkt belirtmek bu durum bizim daha çok aşina olduğumuz klasik python fonksiyonunlarında da gördüğümüz bir durum

In [0]:
# .option("header","True").option("inferSchema","True").load("dbfs:/FileStore/tables/grades.csv")
df = spark.read.csv("dbfs:/FileStore/tables/grades.csv",header=True,inferSchema=True)
df.show()
df.printSchema()

+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|Last name|    "First name"|               "SSN"|        "Test1"| "Test2"| "Test3"| "Test4"| "Final"| "Grade"|
+---------+----------------+--------------------+---------------+--------+--------+--------+--------+--------+
|  Alfalfa|      "Aloysius"|       "123-45-6789"|           40.0|    90.0|   100.0|    83.0|    49.0|    "D-"|
|   Alfred|    "University"|       "123-12-1234"|           41.0|    97.0|    96.0|    97.0|    48.0|    "D+"|
|    Gerty|        "Gramma"|       "567-89-0123"|           41.0|    80.0|    60.0|    40.0|    44.0|     "C"|
|  Android|      "Electric"|       "087-65-4321"|           42.0|    23.0|    36.0|    45.0|    47.0|    "B-"|
|  Bumpkin|          "Fred"|       "456-78-9012"|           43.0|    78.0|    88.0|    77.0|    45.0|    "A-"|
|   Rubble|         "Betty"|       "234-56-7890"|           44.0|    90.0|    80.0|    90.0|    46.0|    "C-"|
|

Eğer sadece istediğimiz bir kolonun değerlerine ihtiyacimiz varsa `select` içine yazarız.

In [0]:
df.select("Last name").show()

+---------+
|Last name|
+---------+
|  Alfalfa|
|   Alfred|
|    Gerty|
|  Android|
|  Bumpkin|
|   Rubble|
|   Noshow|
|     Buff|
|  Airpump|
|   Backus|
|Carnivore|
|    Dandy|
| Elephant|
| Franklin|
|   George|
|Heffalump|
+---------+



Birden fazla kolon için sadece virgülle ayırmak yeterli

In [0]:
df.select("Last name"," \"Final\"").show()


+---------+--------+
|Last name| "Final"|
+---------+--------+
|  Alfalfa|    49.0|
|   Alfred|    48.0|
|    Gerty|    44.0|
|  Android|    47.0|
|  Bumpkin|    45.0|
|   Rubble|    46.0|
|   Noshow|    43.0|
|     Buff|    50.0|
|  Airpump|    83.0|
|   Backus|    97.0|
|Carnivore|    40.0|
|    Dandy|    45.0|
| Elephant|    77.0|
| Franklin|    90.0|
|   George|     4.0|
|Heffalump|    40.0|
+---------+--------+



In [0]:
df = df.withColumnRenamed(" \"Final\"","Final")
df = df.withColumnRenamed("Last name","Last_name")

df.show()
df.printSchema()

+---------+----------------+--------------------+---------------+--------+--------+--------+-----+--------+
|Last_name|    "First name"|               "SSN"|        "Test1"| "Test2"| "Test3"| "Test4"|Final| "Grade"|
+---------+----------------+--------------------+---------------+--------+--------+--------+-----+--------+
|  Alfalfa|      "Aloysius"|       "123-45-6789"|           40.0|    90.0|   100.0|    83.0| 49.0|    "D-"|
|   Alfred|    "University"|       "123-12-1234"|           41.0|    97.0|    96.0|    97.0| 48.0|    "D+"|
|    Gerty|        "Gramma"|       "567-89-0123"|           41.0|    80.0|    60.0|    40.0| 44.0|     "C"|
|  Android|      "Electric"|       "087-65-4321"|           42.0|    23.0|    36.0|    45.0| 47.0|    "B-"|
|  Bumpkin|          "Fred"|       "456-78-9012"|           43.0|    78.0|    88.0|    77.0| 45.0|    "A-"|
|   Rubble|         "Betty"|       "234-56-7890"|           44.0|    90.0|    80.0|    90.0| 46.0|    "C-"|
|   Noshow|         "Cecil"|

In [0]:
df.select("Last_name","Final").show()

+---------+-----+
|Last_name|Final|
+---------+-----+
|  Alfalfa| 49.0|
|   Alfred| 48.0|
|    Gerty| 44.0|
|  Android| 47.0|
|  Bumpkin| 45.0|
|   Rubble| 46.0|
|   Noshow| 43.0|
|     Buff| 50.0|
|  Airpump| 83.0|
|   Backus| 97.0|
|Carnivore| 40.0|
|    Dandy| 45.0|
| Elephant| 77.0|
| Franklin| 90.0|
|   George|  4.0|
|Heffalump| 40.0|
+---------+-----+

